In [3]:
import pandas as pd
import numpy as np
import lda
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool
import os

In [39]:
data_list = []
directory = os.path.join("Data/tweets_by_date/")
for root,dirs,files in os.walk(directory):
    for file in files:
       if file.endswith(".csv"):
           dataframe = pd.read_csv("Data/tweets_by_date/" + file)
           data_list.append(dataframe.values.tolist())


In [58]:
len(data_list[1])

135

In [72]:
Data = []
for i in data_list:
                Data.append(''.join("".join(e) for e in i))

In [73]:
len(Data)

26

In [75]:
X = Data

In [76]:
n_topics =5 # number of topics
n_iter = 500 # number of iterations

# vectorizer: ignore English stopwords & words that occur less than 5 times
cvectorizer = CountVectorizer(min_df=3, stop_words='english')
cvz = cvectorizer.fit_transform(X)

# train an LDA model
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)

INFO:lda:n_documents: 26
INFO:lda:vocab_size: 441
INFO:lda:n_words: 3066
INFO:lda:n_topics: 5
INFO:lda:n_iter: 500
/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -27664
INFO:lda:<10> log likelihood: -20469
INFO:lda:<20> log likelihood: -20082
INFO:lda:<30> log likelihood: -19999
INFO:lda:<40> log likelihood: -19977
INFO:lda:<50> log likelihood: -19916
INFO:lda:<60> log likelihood: -19951
INFO:lda:<70> log likelihood: -19946
INFO:lda:<80> log likelihood: -19805
INFO:lda:<90> log likelihood: -19814
INFO:lda:<100> log likelihood: -19855
INFO:lda:<110> log likelihood: -19882
INFO:lda:<120> log likelihood: -19958
INFO:lda:<130> log likelihood: -19954
INFO:lda:<140> log likelihood: -19893
INFO:lda:<150>

In [77]:
# a t-SNE model
# angle value close to 1 means sacrificing accuracy for speed
# pca initializtion usually leads to better results 
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')

# 20-D -> 2-D
tsne_lda = tsne_model.fit_transform(X_topics)

[t-SNE] Computing 25 nearest neighbors...
[t-SNE] Indexed 26 samples in 0.001s...
[t-SNE] Computed neighbors for 26 samples in 0.211s...
[t-SNE] Computed conditional probabilities for sample 26 / 26
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 45.965565
[t-SNE] Error after 950 iterations: 0.579410


In [78]:
n_top_words = 5 # number of keywords we show

# 20 colors
colormap = np.array(["#1f77b4", "#FF0000", '#6AA84F', '#E69138', '#A64D79'])#, 

In [79]:
_lda_keys = []
for i in range(X_topics.shape[0]):
  _lda_keys +=  X_topics[i].argmax(),

In [80]:
topic_summaries = []
topic_word = lda_model.topic_word_  # all topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
  topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1] # get!
  topic_summaries.append(' '.join(topic_words)) # append!

In [81]:
topic_word = lda_model.topic_word_ 

In [82]:
threshold = 0.5
_idx = np.amax(X_topics, axis=1) > threshold  # idx of doc that above the threshold
X_topics = X_topics[_idx]

In [83]:
topic_summaries

['erdogan says minister man syria',
 'world live government ankara day',
 'like need new arrested understand',
 'media peaceful social video international',
 'police gas people tear square']

In [85]:
_lda_keys

[4, 4, 1, 4, 3, 3, 0, 1, 4, 1, 0, 4, 0, 0, 0, 4, 4, 4, 0, 0, 0, 1, 1, 1, 0, 2]

In [107]:
title = 'CF pooling LDA viz'
num_example = len(X_topics)

plot_lda = bp.figure(plot_width=1400, plot_height=1100,
                     title=title,
                     tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

#source = ColumnDataSource(data=dict(x=a_list, y=an_array))
data_dict = dict(x=tsne_lda[:, 0], y=tsne_lda[:, 1])
data = pd.DataFrame(data_dict, columns = ['content', 'topic'])

source = bp.ColumnDataSource(data=data)

plot_lda.scatter(x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                 color=colormap[_lda_keys][:num_example])
#plot_lda.scatter(x='x', y='y',
#                 color=colormap[_lda_keys][:num_example], source=source)

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('line_color', 3), ('x', 26), ('y', 26)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('fill_color', 3), ('line_color', 3), ('x', 26), ('y', 26)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


bokeh.models.renderers.GlyphRenderer(
    id='a9cc65e7-0d29-4bff-916b-4641ec7e4dba',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='49bd5b5d-319f-4142-93ac-7df8b76dd5d3',
        callback=None,
        data={'fill_color': ['#A64D79', '#A64D79', '#FF0000'],
         'line_color': ['#A64D79', '#A64D79', '#FF0000'],
         'x': {'__ndarray__': '5qOVwlXEJcLTahNDQmhDQ4WJ7kJrTqnC8yycQFBUFMNcsRBCdEAtwQ1oGsNwiXpCWab7wpfbIUTkkovCof3iQn+ESUILAEXDFJe1wluwjUJdxy1CajrRwMzR78EIpgFDc5Dhwj+fkkE=',
          'dtype': 'float32',
          'shape': (26,)},
         'y': {'__ndarray__': 'ZUciQnSQMkNq4ijCGb4XQtTbUEKQB+TBQWrRwlX5K0G2hVtDj7UHwsazocK7VErAx6owQ8acvkKzvMTC8AwIw83njUI8GZFChCIrw472kMK2aAxDnO4GQmm/z0LBogpDpFLHQr+/M8M=',
          'dtype': 'float32',
          'shape': (26,)}},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='9579687c-8d0b-4a1c-baa5-8751aad07f90',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='7b610584-0b7a-4a2a-ab83-c1f2f3512c2c',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.markers.Circle(
        id='b2690643-d83f-4880-b310-42b1edbfb7b4',
        angle={'units': 'rad', 'value': 0.0},
        fill_alpha={'value': 1.0},
        fill_color={'field': 'fill_color'},
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha={'value': 1.0},
        line_cap='butt',
        line_color={'field': 'line_color'},
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width={'value': 1},
        name=None,
        radius=None,
        radius_dimension='x',
        size={'units': 'screen', 'value': 4},
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph=bokeh.models.markers.Circle(
        id='d8e99281-4b74-46d7-80bb-6e5f25b228c2',
        angle={'units': 'rad', 'value': 0.0},
        fill_alpha={'value': 0.1},
        fill_color={'value': '#1f77b4'},
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha={'value': 0.1},
        line_cap='butt',
        line_color={'value': '#1f77b4'},
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width={'value': 1},
        name=None,
        radius=None,
        radius_dimension='x',
        size={'units': 'screen', 'value': 4},
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    selection_glyph=None,
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='61923b3e-1739-4b25-949c-fdf26a3aa36c',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='49bd5b5d-319f-4142-93ac-7df8b76dd5d3',
            callback=None,
            data={'fill_color': ['#A64D79', '#A64D79', '#FF0000'],
             'line_color': ['#A64D79', '#A64D79', '#FF0000'],
             'x': {'__ndarray__': '5qOVwlXEJcLTahNDQmhDQ4WJ7kJrTqnC8yycQFBUFMNcsRBCdEAtwQ1oGsNwiXpCWab7wpfbIUTkkovCof3iQn+ESUILAEXDFJe1wluwjUJdxy1CajrRwMzR78EIpgFDc5Dhwj+fkkE=',
              'dtype': 'float32',
              'shape': (26,)},
             'y': {'__ndarray__': 'ZUciQnSQMkNq4ijCGb4XQtTbUEKQB+TBQWrRwlX5K0G2hVtDj7UHwsazocK7VErAx6owQ8a

In [108]:
# randomly choose a news (within a topic) coordinate as the crucial words coordinate
topic_coord = np.empty((X_topics.shape[1], 2)) * np.nan
for topic_num in _lda_keys:
  if not np.isnan(topic_coord).any():
    break
  topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

# plot crucial words
for i in range(X_topics.shape[1]):
  plot_lda.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key"}

# save the plot
save(plot_lda, '{}.html'.format(title))

/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/bokeh/io/saving.py:123: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/fatma/anaconda2/envs/py35/lib/python3.5/site-packages/bokeh/io/saving.py:136: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/fatma/PhD/Git/semi_supervision/semi_supervision/CF pooling LDA viz.html'